In [2]:
import cbor2
import numpy as np
from PIL import Image

In [3]:
def image_to_float_matrix(image_path):
    # 打开图片并转换为灰度模式
    img = Image.open(image_path).convert('L')  # 'L' 是灰度模式

    # 将图片数据转换为 NumPy 数组
    img_array = np.array(img, dtype=np.float64)

    # 将像素值从 [0, 255] 范围映射到 [0, 1] 范围
    img_array = 1 - img_array/255.0

    return img_array

In [4]:
image_path = "probe.png"
img_matrix = image_to_float_matrix("probe.png")
# 获取图像的宽度和高度
height, width = img_matrix.shape
# 将图像矩阵展平为1D数组
cells = img_matrix.flatten()
image_data = {
    "width": width,
    "height": height,
    "cells": cells  # 转换为 Python 列表，以便存储为 CBOR
}
print(image_data)

{'width': 388, 'height': 374, 'cells': array([0.00392157, 0.00392157, 0.00392157, ..., 0.00392157, 0.00392157,
       0.07058824])}


In [5]:
def get_block_data(image_data):
    # 计算每个块的大小
    block_size_x = image_data['width'] // 26
    block_size_y = image_data['height'] // 25
    primary_x = [i * 388 // 26 for i in range(26)] + [388]
    primary_y = [i * 374 // 25 for i in range(25)] + [374]
    
    block_data = {
        "pixels": {
            "x": image_data['width'],
            "y": image_data['height']
        },
        "primary": {
            "blocks": {
                "x": 26,
                "y": 25
            },
            "corners": {
                "x": 27,
                "y": 26
            },
            "x": primary_x,
            "y": primary_y
        },
        "secondary": {
            "blocks": {
                "x": 27,
                "y": 26
            },
            "corners": {
                "x": 28,
                "y": 27
            },
            "x": [0] + [(primary_x[i] + primary_x[i+1])//2 for i in range(25 + 1)] + [388],
            "y": [0] + [(primary_y[i] + primary_y[i+1])//2 for i in range(24 + 1)] + [374]
        }
    }
    return block_data

In [6]:
block_data = get_block_data(image_data)
print(len(block_data['primary']['x']))
print(block_data['primary']['x'])
print(len(block_data['primary']['y']))
print(block_data['primary']['y'])
print(len(block_data['secondary']['x']))
print(block_data['secondary']['x'])
print(len(block_data['secondary']['y']))
print(block_data['secondary']['y'])

27
[0, 14, 29, 44, 59, 74, 89, 104, 119, 134, 149, 164, 179, 194, 208, 223, 238, 253, 268, 283, 298, 313, 328, 343, 358, 373, 388]
26
[0, 14, 29, 44, 59, 74, 89, 104, 119, 134, 149, 164, 179, 194, 209, 224, 239, 254, 269, 284, 299, 314, 329, 344, 359, 374]
28
[0, 7, 21, 36, 51, 66, 81, 96, 111, 126, 141, 156, 171, 186, 201, 215, 230, 245, 260, 275, 290, 305, 320, 335, 350, 365, 380, 388]
27
[0, 7, 21, 36, 51, 66, 81, 96, 111, 126, 141, 156, 171, 186, 201, 216, 231, 246, 261, 276, 291, 306, 321, 336, 351, 366, 374]


In [7]:
img = Image.open(image_path).convert('L')  # 'L' 是灰度模式
img_array = 255 - np.array(img, dtype=np.float64)

counts = []
for i in range(block_data['primary']['blocks']['x']):
    tmp_data = []
    for j in range(block_data['primary']['blocks']['y']):
        region = np.array(img_array)[block_data['primary']['y'][j]:block_data['primary']['y'][j+1], block_data['primary']['x'][i]:block_data['primary']['x'][i+1]]
        # 使用 numpy.histogram 计算直方图
        hist, bins = np.histogram(region.flatten(), bins=256, range=[0, 256])
        tmp_data.append(hist)
    counts.append(tmp_data)
counts = np.array(counts, dtype=np.int32)
hist_data = {
    "width": block_data['primary']['blocks']['x'],
    "height": block_data['primary']['blocks']['y'],
    "bins": 256,
    "counts": counts.flatten()
}
print(counts.shape)


(26, 25, 256)


## 平滑直方图
根据四个次块的附近四个主块，对直方图求和。数值范围[0, 900]

In [9]:
smoothed_counts = []
for i in range(block_data['secondary']['blocks']['x']):
    tmp_data = []
    for j in range(block_data['secondary']['blocks']['y']):
        sum_count = None
        if i-1 >= 0 and j-1 >= 0 and i <= block_data['primary']['blocks']['x']-1 and j <= block_data['primary']['blocks']['y']-1:
            sum_count = counts[i-1][j-1] + counts[i][j-1] + counts[i-1][j] + counts[i][j]
        # 四个角
        elif (i-1 < 0 and j-1 < 0): 
            sum_count = counts[i][j]
        elif (i-1 < 0 and j > block_data['primary']['blocks']['y']-1): 
            sum_count = counts[i][j-1]
        elif (j-1 < 0 and i > block_data['primary']['blocks']['x']-1): 
            sum_count = counts[i-1][j]
        elif (i > block_data['primary']['blocks']['x']-1 and j > block_data['primary']['blocks']['y']-1): 
            sum_count = counts[i-1][j-1]
        # 四个边
        elif (i-1 < 0 and j <= block_data['primary']['blocks']['y']-1):
            sum_count = counts[i][j-1] + counts[i][j]
        elif (i > block_data['primary']['blocks']['x']-1 and j <= block_data['primary']['blocks']['y']-1):
            sum_count = counts[i-1][j-1] + counts[i-1][j]
        elif (i <= block_data['primary']['blocks']['x']-1 and j-1 < 0):
            sum_count = counts[i-1][j] + counts[i-1][j]
        elif (i <= block_data['primary']['blocks']['x']-1 and j > block_data['primary']['blocks']['y']-1):
            sum_count = counts[i-1][j-1] + counts[i][j-1]
        tmp_data.append(sum_count)
    smoothed_counts.append(tmp_data)
smoothed_counts = np.array(smoothed_counts, dtype=np.int32)
smoothed_hist_data = {
    "width": block_data['secondary']['blocks']['x'],
    "height": block_data['secondary']['blocks']['y'],
    "bins": 256,
    "smoothed_counts": smoothed_counts.flatten()
}
print(smoothed_hist_data)

{'width': 27, 'height': 26, 'bins': 256, 'smoothed_counts': array([  0, 196,   0, ...,   0,   0,   0], dtype=int32)}


## 对比度
计算当前块的直方图总和→计算上下界→对比度值是 (upperBound - lowerBound) 除以直方图的总区间长度 (histogram.bins - 1)


In [11]:
# 对主块平滑直方图
primary_smoothed_counts = []
for i in range(block_data['primary']['blocks']['x']):
    tmp_data = []
    for j in range(block_data['primary']['blocks']['y']):
        sum_count = smoothed_counts[i][j] + smoothed_counts[i+1][j] + smoothed_counts[i][j+1] + smoothed_counts[i+1][j+1]
        tmp_data.append(sum_count)
    primary_smoothed_counts.append(tmp_data)
primary_smoothed_counts = np.array(primary_smoothed_counts, dtype=np.int32)
primary_smoothed_hist_data = {
    "width": block_data['primary']['blocks']['x'],
    "height": block_data['primary']['blocks']['y'],
    "bins": 256,
    "primary_smoothed_counts": primary_smoothed_counts.flatten()
}
print(primary_smoothed_hist_data)

{'width': 26, 'height': 25, 'bins': 256, 'primary_smoothed_counts': array([   0, 1835,    0, ...,    0,    0,    0], dtype=int32)}


In [12]:
res_list = []
CLIPPED_CONTRAST = 0.08
for i in range(block_data['primary']['blocks']['x']):
    for j in range(block_data['primary']['blocks']['y']):
        volume = sum(counts[i][j])
        clipLimit = round(volume * CLIPPED_CONTRAST)
        # 下界
        accumulator = 0
        lowerBound = 256 - 1
        for k in range(256):
            accumulator += counts[i][j][k]
            if accumulator > clipLimit:
                lowerBound = k
                break
        # 上界
        accumulator = 0
        upperBound = 0
        for k in range(256 - 1, -1, -1):
            accumulator += counts[i][j][k]
            if accumulator > clipLimit:
                upperBound = k
                break
        res = (upperBound - lowerBound) / (256.0 - 1)
        res_list.append(res)
contrast_data = {
    "width": block_data['primary']['blocks']['x'],
    "height": block_data['primary']['blocks']['y'],
    "cells": res_list
}
contrast_data

{'width': 26,
 'height': 25,
 'cells': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.023529411764705882,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.1568627450980392,
  0.1411764705882353,
  0.3764705882352941,
  0.5176470588235295,
  0.30980392156862746,
  0.4235294117647059,
  0.4745098039215686,
  0.40784313725490196,
  0.42745098039215684,
  0.26666666666666666,


## 绝对对比度蒙版

In [14]:
def filter_absolute_contrast(contrast_list):
    threshold = 0.06666666666666667
    result = [value < threshold for value in contrast_list]
    return result
absolute_contrast_mask = filter_absolute_contrast(contrast_data['cells'])
absolute_contrast_data = {
    "width": block_data['primary']['blocks']['x'],
    "height": block_data['primary']['blocks']['y'],
    "cells": absolute_contrast_mask
}

## 相对对比度蒙版

In [16]:
def filter_relative_contrast(contrast, blocks):
    sorted_contrast = contrast.copy()
    sorted_contrast.sort(reverse=True)
    pixels_per_block = blocks['pixels']['x'] * blocks['pixels']['y'] / (blocks['primary']['blocks']['x'] * blocks['primary']['blocks']['y'])
    sample_count = min(len(sorted_contrast), 168568 // pixels_per_block)
    considered_blocks = max(round(sample_count * 0.49), 1)
    average_contrast = np.mean(sorted_contrast[:considered_blocks])
    limit = average_contrast * 0.34
    result = contrast < limit
    return result
relative_contrast_mask = filter_relative_contrast(contrast_data['cells'], block_data)
relative_contrast_data = {
    "width": block_data['primary']['blocks']['x'],
    "height": block_data['primary']['blocks']['y'],
    "cells": relative_contrast_mask
}

## 组合掩码

In [18]:
def merge1(contrast1, contrast2):
    return [bool(a or b) for a, b in zip(contrast1, contrast2)]
combined_mask = merge1(absolute_contrast_data['cells'], relative_contrast_data['cells'])
combined_data = {
    "width": block_data['primary']['blocks']['x'],
    "height": block_data['primary']['blocks']['y'],
    "cells": combined_mask
}

## 过滤掩码

In [20]:
# int矩形
class IntRect:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        
    def left(self):
        return self.x
        
    def right(self):
        return self.x + self.width
        
    def top(self):
        return self.y
        
    def bottom(self):
        return self.y + self.height
        
    def area(self):
        self.width * self.height

# boolean矩阵
class BooleanMatrix:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.cells = [0 for i in range(self.width * self.height)]
    
    def get(self, x, y):
        return self.cells[self.offset(x, y)]
    
    def offset(self, x, y):
        return y * self.width + x
    
    def set(self, x, y, value):
        self.cells[self.offset(x, y)] = value

    def invert(self):
        for i in range(len(self.cells)):
            self.cells[i] = 1 if self.cells[i] == 0 else 0
            
    def setCells(self, cells):
        for i in range(len(self.cells)):
            self.cells[i] = cells[i]
            
    def merge(self, other):
        for i in range(len(self.cells)):
            self.cells[i] |= other.cells[i]

# int矩阵
class IntMatrix:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.array = [0 for i in range(self.width * self.height)]

    def get(self, x, y):
        return self.array[self.offset(x, y)]
    
    def offset(self, x, y):
        return y * self.width + x
    
    def set(self, x, y, value):
        self.array[self.offset(x, y)] = value
        

In [21]:
import math

def filterBlockErrors(input):
    return vote(input, mask=None, radius=1, majority=0.7, border_distance=4)


def vote(input, mask, radius, majority, border_distance):
    size = {'x': input.width, 'y': input.height}
    rect = IntRect(border_distance, border_distance, size['x'] - 2 * border_distance, size['y'] - 2 * border_distance)
    thresholds = [math.ceil(majority * i) for i in range((2 * radius + 1) ** 2 + 1)]

    counts = IntMatrix(size['x'],size['y'])
    output = BooleanMatrix(size['x'],size['y'])

    for y in range(rect.top(), rect.bottom()):
        super_top = y - radius - 1
        super_bottom = y + radius
        y_min = max(0, y - radius)
        y_max = min(size['y'] - 1, y + radius)
        y_range = y_max - y_min + 1

        for x in range(rect.left(), rect.right()):
            if mask is None or mask.get(x, y):
                left = counts.get(x-1, y) if x > 0 else 0
                top = counts.get(x, y - 1) if y > 0 else 0
                diagonal = counts.get(x - 1, y - 1) if (x > 0 and y > 0) else 0
                x_min = max(0, x - radius)
                x_max = min(size['x'] - 1, x + radius)
                if left > 0 and top > 0 and diagonal > 0:
                    ones = top + left - diagonal - 1
                    ny = x - radius - 1
                    super_right = x + radius
                    if ny >= 0 and super_top >= 0 and input.get(ny, super_top):
                        ones += 1
                    if ny >= 0 and super_bottom < size['y'] and input.get(ny, super_bottom):
                        ones -= 1
                    if super_right < size['x'] and super_top >= 0 and input.get(super_right, super_top):
                        ones -= 1
                    if super_right < size['x'] and super_bottom < size['y'] and input.get(super_right, super_bottom):
                        ones += 1
                else:
                    ones = 0
                    for ny in range(y_min, y_max + 1):
                        for super_right in range(x_min, x_max + 1):
                            if input.get(super_right, ny):
                                ones += 1

                counts.set(x, y, ones + 1)
                if ones >= thresholds[y_range * (x_max - x_min + 1)]:
                    output.set(x, y, True)

    return output

In [22]:
# 初始化标准结构
mask = BooleanMatrix(combined_data['width'], combined_data['height'])
mask.setCells(combined_data['cells'])
# 过滤蒙版
mask.merge(filterBlockErrors(mask))
mask.invert()
mask.merge(filterBlockErrors(mask))
mask.merge(filterBlockErrors(mask))
mask.merge(vote(mask, None, radius = 7, majority=0.51, border_distance=4))

## Equalized image 均衡图像